## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-14-22-50-53 +0000,nyt,What Hurricane Models Say About Tropical Storm...,https://www.nytimes.com/2025/08/14/weather/hur...
1,2025-08-14-22-49-06 +0000,nyt,"Gerry Spence, Courtroom Showman in Buckskin, D...",https://www.nytimes.com/2025/08/14/us/gerry-sp...
2,2025-08-14-22-49-00 +0000,wsj,The U.S. Is Discussing Taking a Stake in Intel,https://www.wsj.com/tech/us-stake-intel-1ff245...
3,2025-08-14-22-48-00 +0000,nypost,Nike co-founder Phil Knight and wife pledge re...,https://nypost.com/2025/08/14/us-news/nike-co-...
4,2025-08-14-22-41-42 +0000,nyt,All National Guard Troops Sent to Washington A...,https://www.nytimes.com/2025/08/14/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2320/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
47,trump,58
63,putin,23
146,up,12
58,russia,12
505,new,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
206,2025-08-14-11-58-04 +0000,nypost,Russia appears ready to test new missile — eve...,https://nypost.com/2025/08/14/world-news/russi...,125
158,2025-08-14-15-59-01 +0000,nypost,Trump reveals the odds of Putin summit ending ...,https://nypost.com/2025/08/14/us-news/trump-gi...,122
8,2025-08-14-22-33-36 +0000,nyt,Russia and Ukraine Agree: A Trump Summit Is a ...,https://www.nytimes.com/2025/08/14/world/europ...,116
114,2025-08-14-17-56-02 +0000,nypost,India shops the globe for oil after 50% tariff...,https://nypost.com/2025/08/14/world-news/india...,114
200,2025-08-14-12-44-56 +0000,nypost,"Putin heaps praise on ‘sincere’ Trump, hints R...",https://nypost.com/2025/08/14/us-news/putin-pr...,113


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
206,125,2025-08-14-11-58-04 +0000,nypost,Russia appears ready to test new missile — eve...,https://nypost.com/2025/08/14/world-news/russi...
278,44,2025-08-14-03-00-00 +0000,wsj,As other Asian countries ramp up manufacturing...,https://www.wsj.com/economy/trade/trump-tariff...
13,43,2025-08-14-22-26-00 +0000,wsj,California Democrats are planning their own re...,https://www.wsj.com/politics/policy/gavin-news...
107,42,2025-08-14-18-15-15 +0000,nypost,Air Canada warns it will cancel 500 flights by...,https://nypost.com/2025/08/14/business/air-can...
140,41,2025-08-14-16-29-11 +0000,nypost,DC cops allowed to alert ICE about illegal imm...,https://nypost.com/2025/08/14/us-news/dc-cops-...
172,35,2025-08-14-14-49-42 +0000,cbc,Former justice minister Irwin Cotler calls on ...,https://www.cbc.ca/news/canada/montreal/irwin-...
114,33,2025-08-14-17-56-02 +0000,nypost,India shops the globe for oil after 50% tariff...,https://nypost.com/2025/08/14/world-news/india...
40,32,2025-08-14-21-36-51 +0000,startribune,State grand jury indicts Vance Boelter on firs...,https://www.startribune.com/hennepin-county-gr...
31,32,2025-08-14-21-51-42 +0000,nypost,"3-year-old breaks $1,600 marble table at NJ ca...",https://nypost.com/2025/08/14/us-news/nj-mom-c...
47,30,2025-08-14-21-21-59 +0000,nypost,Chicago voters stuck with ‘America’s Worst May...,https://nypost.com/2025/08/14/us-news/chicago-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
